In [1]:
!pip install torch torchvision
!pip install Pillow
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 96.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 52.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [6]:
import torch
import clip
from PIL import Image
import requests
from io import BytesIO

In [3]:
def get_caption_from_file(image_path, candidate_file, model_name="ViT-B/32"):
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # Load the CLIP model and its preprocessing function.
    model, preprocess = clip.load(model_name, device=device)

    # Load and preprocess the image.
    image = Image.open(image_path)
    image_input = preprocess(image).unsqueeze(0).to(device)

    # Read candidate captions from file (one caption per line).
    with open(candidate_file, "r", encoding="utf-8") as f:
        candidate_captions = [line.strip() for line in f if line.strip()]

    # Tokenize the candidate captions.
    text_inputs = clip.tokenize(candidate_captions).to(device)

    # Compute the image and text features.
    with torch.no_grad():
        image_features = model.encode_image(image_input)
        text_features = model.encode_text(text_inputs)

    # Normalize the features to get cosine similarity as the dot product.
    image_features = image_features / image_features.norm(dim=-1, keepdim=True)
    text_features = text_features / text_features.norm(dim=-1, keepdim=True)

    # Compute cosine similarity between the image and each candidate caption.
    similarities = (image_features @ text_features.T).squeeze(0)

    # Select the caption with the highest similarity.
    best_caption_index = torch.argmax(similarities).item()
    best_caption = candidate_captions[best_caption_index]

    return best_caption

In [7]:
def chat():
    image_url = input("Please enter the image URL: ").strip()
    try:
        response = requests.get(image_url)
        response.raise_for_status()
        image_data = BytesIO(response.content)
        return image_data
    except Exception as e:
        print("Failed to retrieve the image. Error:", e)
        exit(1)

In [9]:
if __name__ == "__main__":
  while(True):
    image_source = chat()
    candidate_file = "candidate_captions.txt"

    caption = get_caption_from_file(image_source, candidate_file)
    print("Selected Caption:", caption)

    print("Would you like to continue? (y/n)")
    choice = input().strip().lower()
    if choice != 'y':
        print("Bye 👋")
        break

Please enter the image URL: https://th.bing.com/th/id/OIP.vlFp_jWM3fklHxAHuX1MyAHaEo?rs=1&pid=ImgDetMain
Selected Caption: A peaceful lake surrounded by mountains.
Would you like to continue? (y/n)
y
Please enter the image URL: https://images.pexels.com/photos/257540/pexels-photo-257540.jpeg?cs=srgb&dl=pexels-pixabay-257540.jpg&fm=jpg
Selected Caption: A dog playing fetch in a park.
Would you like to continue? (y/n)
n
